<a href="https://colab.research.google.com/github/Anjasfedo/Code-as-a-Cryptography/blob/main/ecc_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# y^2 = x^3 + 7 mod 17
def curve_equation(x, a=0, b=7, p=17):
    """Elliptic curve equation y^2 = x^3 + ax + b mod p."""
    return (x**3 + a*x + b) % p

def is_curve_point(x, y, a=0, b=7, p=17):
    """Check if the point (x, y) satisfies the elliptic curve equation mod p."""
    return curve_equation(x, a, b, p) == (y**2) % p

In [21]:
def mod_inverse(a, p):
    """Returns the modular inverse of a under modulo p using Fermat's Little Theorem."""
    return pow(a, p - 2, p)

def calculate_slope_mod(P, Q, p, a):
    x1, y1 = P
    x2, y2 = Q
    if x1 == x2 and y1 == y2:
        # Use the formula for point doubling when P == Q
        numerator = (3 * x1**2 + a) % p
        denominator = (2 * y1) % p
    else:
        # Use the formula for regular slope when P != Q
        numerator = (y2 - y1) % p
        denominator = (x2 - x1) % p

    if denominator == 0:
        raise ValueError("Slope is undefined (denominator is zero).")

    # Compute the slope as (numerator / denominator) % p, which is
    # numerator * mod_inverse(denominator, p) % p
    slope = (numerator * mod_inverse(denominator, p)) % p
    return slope

# Point Addition

In [22]:
def calculate_add_xr_mod(P, Q, m, p):
    x1, y1 = P
    x2, y2 = Q
    xr = (m**2 - x1 - x2) % p
    return xr

In [23]:
def calculate_add_yr_mod(P, Q, m, xr, p):
    x1, y1 = P
    x2, y2 = Q
    yr = (m * (x1 - xr) - y1) % p
    return yr

In [24]:
def calculate_point_add(P, Q, p, a):
  print(f'P: {P}')
  print(f'Q: {Q}')
  print(f'p: {p}')

  m = calculate_slope_mod(P, Q, p, a)
  print(f'slope: {m}')

  xr = calculate_add_xr_mod(P, Q, m, p)
  print(f'xr: {xr}')

  yr = calculate_add_yr_mod(P, Q, m, xr, p)
  print(f'yr: {yr}')

  R = (xr, yr)
  print(f'R: {R}')

  print(f'is R on curve: {is_curve_point(R[0], R[1], p)}')

  return R

# Point Multiplication

In [25]:
def calculate_mul_xr_mod(P, Q, slope, p):
    x1, y1 = P
    x2, y2 = Q
    xr = (slope**2 - (2 * x1)) % p
    return xr

In [26]:
def calculate_mul_yr_mod(P, Q, slope, xr, p):
    x1, y1 = P
    yr = (slope * (x1 - xr) - y1) % p
    return yr

In [27]:
def calculate_point_mul(P, p, a):
  print(f'P: {P}')
  print(f'p: {p}')

  m = calculate_slope_mod(P, P, p, a)
  print(f'slope: {m}')

  xr = calculate_mul_xr_mod(P, P, m, p)
  print(f'xr: {xr}')

  yr = calculate_mul_yr_mod(P, P, m, xr, p)
  print(f'yr: {yr}')

  R = (xr, yr)
  print(f'R: {R}')

  print(f'is R on curve: {is_curve_point(R[0], R[1], p)}')

  return R

# Test

In [28]:
# y^2 = x^3 + x + 6
def curve_equation(x, p, a=1, b=6):
    """Elliptic curve equation y^2 = x^3 + ax + b mod p."""
    return (x**3 + a*x + b) % p

def is_curve_point(x, y, p=11):
    """Check if the point (x, y) satisfies the elliptic curve equation mod p."""
    return curve_equation(x, p=p) == (y**2) % p

In [29]:
P = (2, 4)
Q = (5, 9)

R = calculate_point_add(P, Q, 11, 1)

P: (2, 4)
Q: (5, 9)
p: 11
slope: 9
xr: 8
yr: 8
R: (8, 8)
is R on curve: True


In [30]:
_R = calculate_point_mul(P, 11, 1)

P: (2, 4)
p: 11
slope: 3
xr: 5
yr: 9
R: (5, 9)
is R on curve: True


# Elliptic Curve Diffie Hellman

In [31]:
def curve_equation(x, p, a=2, b=1):
    """Elliptic curve equation y^2 = x^3 + ax + b mod p."""
    return (x**3 + a*x + b) % p

def is_curve_point(x, y, p=5):
    """Check if the point (x, y) satisfies the elliptic curve equation mod p."""
    return curve_equation(x, p=p) == (y**2) % p

In [32]:
# y^2 = x^3 + 2x + 1
a = 2
# b = 1

p = 5
P1 = (0, 1)

PA = calculate_point_mul(P, p, a)

P: (2, 4)
p: 5
slope: 3
xr: 0
yr: 2
R: (0, 2)
is R on curve: False


In [33]:
PB = calculate_point_add(P1, PA, p, a)

P: (0, 1)
Q: (0, 2)
p: 5


ValueError: Slope is undefined (denominator is zero).

In [ ]:
KA = calculate_point_mul(PB, p, a)

In [ ]:
KB_1 = calculate_point_mul(PA, p, a)

In [ ]:
KB_2 = calculate_point_add(P ,KB_1, p, a)

In [ ]:
KB_3 = calculate_point_add(P, KB_2, p, a)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to plot a generalized elliptic curve with a callback equation
def plot_elliptic_curve(a, b, equation_callback, x_range=(-10, 10), num_points=100000, curve_label=None):
    """
    Plot an elliptic curve given an equation as a callback.

    Parameters:
    a (float): The 'a' coefficient to pass into the elliptic curve equation.
    b (float): The 'b' coefficient to pass into the elliptic curve equation.
    equation_callback (function): A callback function defining the elliptic curve equation.
    x_range (tuple): The range of x values for plotting (default is from -10 to 10).
    num_points (int): The number of points to plot (default is 100000).
    curve_label (str): A custom label for the curve plot (default uses the equation).
    """
    # Generate a set of x values for plotting the elliptic curve
    x = np.linspace(x_range[0], x_range[1], num_points)

    # Calculate the corresponding y^2 values using the provided callback function
    y_squared = equation_callback(x, a, b)

    # Mask values where y^2 is negative (because sqrt can't be applied to negative numbers)
    valid_mask = y_squared >= 0
    y_squared_valid = y_squared[valid_mask]
    x_valid = x[valid_mask]

    # Calculate y values (positive and negative branches) from valid y^2 values
    y_positive = np.sqrt(y_squared_valid)
    y_negative = -np.sqrt(y_squared_valid)

    # If no label is provided, use the default equation format in the label
    if curve_label is None:
        curve_label = rf'$y = \sqrt{{x^3 + {a}x + {b}}}$'

    # Plot the elliptic curve
    plt.figure(figsize=(8, 8))
    plt.plot(x_valid, y_positive, label=curve_label, color='blue')  # Upper half of the curve
    plt.plot(x_valid, y_negative, color='blue')  # Lower half of the curve

    # Set equal aspect ratio to ensure x and y scales are the same
    plt.gca().set_aspect('equal', adjustable='box')

    # Add labels, title, and grid
    plt.title(f"Elliptic Curve: {curve_label}")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)

    plt.xticks(np.arange(-10, 11, 2))
    plt.yticks(np.arange(-10, 11, 2))
    plt.grid(True)
    plt.legend()

    # Show the plot
    plt.show()

# Example callback function for the elliptic curve equation y^2 = x^3 + ax + b
def equation_elliptic_example(x, a, b):
    return x**3 + a*x + b

# Example usage with parameters a = 3, b = 5, and the equation callback
plot_elliptic_curve(a=2, b=1, equation_callback=equation_elliptic_example, curve_label='x**3 + a*x + b')

## Calculate Point

In [37]:
def calculate_slope_mod(P, Q, p):
    """Calculate the slope for two points P and Q on the elliptic curve mod p."""
    x1, y1 = P
    x2, y2 = Q

    if P == Q:
        # Slope for point doubling
        return ((3 * x1**2) * pow(2 * y1, -1, p)) % p
    else:
        # Slope for point addition
        return ((y2 - y1) * pow(x2 - x1, -1, p)) % p

def point_mul_add(P, k, p):
    """Performs scalar multiplication of point P by scalar k and saves each step to an array."""
    current_point = P
    result_point = None  # This will hold the final result
    steps = []  # Array to store each intermediate step

    # Double-and-add algorithm for elliptic curve scalar multiplication
    for i in range(k.bit_length(), -1, -1):
        if result_point is not None:
            # Point doubling step
            slope = calculate_slope_mod(result_point, result_point, p)
            xr = calculate_mul_xr_mod(result_point, result_point, slope, p)
            yr = calculate_mul_yr_mod(result_point, result_point, slope, xr, p)
            result_point = (xr, yr)
            steps.append(('Double', result_point))

        if k & (1 << i):
            if result_point is None:
                result_point = current_point
            else:
                # Point addition step
                slope = calculate_slope_mod(result_point, current_point, p)
                xr = calculate_add_xr_mod(result_point, current_point, slope, p)
                yr = calculate_add_yr_mod(result_point, current_point, slope, xr, p)
                result_point = (xr, yr)
                steps.append(('Add', result_point))

    return steps

# Example usage:
P = (15, 13)
k = 5
p = 17

steps = point_mul_add(P, k, p)

# Print steps
for step in steps:
    print(step)

('Double', (2, 10))
('Double', (12, 1))
('Add', (6, 6))


In [53]:
def calculate_slope_mod(P, Q, a, p):
    """Calculate the slope for two points P and Q on the elliptic curve mod p."""
    x1, y1 = P
    x2, y2 = Q

    if P == Q:
        # Slope for point doubling
        denominator = (2 * y1) % p
        if denominator == 0:
            return None  # This represents the point at infinity
        try:
            inverse_denominator = pow(denominator, -1, p)
        except ValueError:
            raise ValueError(f"Denominator {denominator} is not invertible modulo {p}")
        return ((3 * x1**2 + a) * inverse_denominator) % p
    else:
        # Slope for point addition
        denominator = (x2 - x1) % p
        if denominator == 0:
            return None  # This represents the point at infinity
        try:
            inverse_denominator = pow(denominator, -1, p)
        except ValueError:
            raise ValueError(f"Denominator {denominator} is not invertible modulo {p}")
        return ((y2 - y1) * inverse_denominator) % p

def point_mul_add(P, k, n, p, a):
    """Performs scalar multiplication of point P by scalar k modulo n and returns point or (None, None) at step n."""
    if k == 0:
        return (None, None)
    if k == 1:
        return P  # Return the base point for k = 1

    current_point = P
    steps = []  # Array to store each intermediate step

    k_mod_n = k % n  # Wrap the value of k around the group order

    # Perform first point doubling (multiplication)
    slope = calculate_slope_mod(current_point, current_point, a, p)
    if slope is None:
        return (None, None)  # Point at infinity

    xr = calculate_mul_xr_mod(current_point, current_point, slope, p)
    yr = calculate_mul_yr_mod(current_point, current_point, slope, xr, p)
    result_point = (xr, yr)
    current_point = result_point  # Update current_point to the result

    # Perform point additions for k > 1
    for _ in range(k_mod_n - 2):  # -2 because we already doubled once and counted the base point
        slope = calculate_slope_mod(result_point, P, a, p)
        if slope is None:
            return (None, None)  # Point at infinity

        xr = calculate_add_xr_mod(result_point, P, slope, p)
        yr = calculate_add_yr_mod(result_point, P, slope, xr, p)
        result_point = (xr, yr)
        current_point = result_point  # Update current_point for the next iteration

    return result_point

# Example usage:
P = (0, 1)
n = 7  # This is the order of the curve
p = 5
a = 2  # The coefficient 'a' from the elliptic curve equation y^2 = x^3 + ax + b

# Simulate the behavior for k in range 0 to 25
for k in range(0, 25):
    if k == n:
        # Return (None, None) at step n
        result = (None, None)
    else:
        # Wrap around for k > n
        result = point_mul_add(P, k, n, p, a)
    print(f"{k} * G = {result}")

0 * G = (None, None)
1 * G = (0, 1)
2 * G = (1, 3)
3 * G = (3, 3)
4 * G = (3, 2)
5 * G = (1, 2)
6 * G = (0, 4)
7 * G = (None, None)
8 * G = (1, 3)
9 * G = (1, 3)
10 * G = (3, 3)
11 * G = (3, 2)
12 * G = (1, 2)
13 * G = (0, 4)
14 * G = (1, 3)
15 * G = (1, 3)
16 * G = (1, 3)
17 * G = (3, 3)
18 * G = (3, 2)
19 * G = (1, 2)
20 * G = (0, 4)
21 * G = (1, 3)
22 * G = (1, 3)
23 * G = (1, 3)
24 * G = (3, 3)


In [60]:
# private 1 * public 2 mod
2*3 % 7

6

In [61]:
# private 2 * public 1 mod
3*2 % 7

6

In [63]:
# priv1 = 3
# pub1 = 5

# priv2 = 2
# pub2 = 4

In [66]:
# basis * private = public
# basis * private = public
# G * d = Q

In [73]:
G = 1

# d = private
d1 = 12
d2 = 2

# Q = public
Q1 = d1 * G % 7
Q2 = d2 * G % 7

print(f'Q1: {Q1}')
print(f'Q2: {Q2}')

#

# C = same val
C1 = d1 * Q2 % 7
C2 = d2 * Q1 % 7

print(f'C1: {C1}')
print(f'C2: {C2}')

Q1: 5
Q2: 2
C1: 3
C2: 3


In [74]:
# Generator G and prime modulus p
G = 1
p = 7  # Prime modulus (chosen to be 7 for simplicity)

# d = private keys for two parties
d1 = 12  # Private key for party 1 (Alice)
d2 = 2   # Private key for party 2 (Bob)

# Q = public keys (generated by each party)
Q1 = (d1 * G) % p  # Alice's public key
Q2 = (d2 * G) % p  # Bob's public key

print(f'Q1 (Alice public): {Q1}')
print(f'Q2 (Bob public): {Q2}')

# Each party computes the shared secret
# C = shared secret computed by both parties
C1 = (Q2 * d1) % p  # Alice computes shared secret using Bob's public key
C2 = (Q1 * d2) % p  # Bob computes shared secret using Alice's public key

print(f'C1 (Alice shared secret): {C1}')
print(f'C2 (Bob shared secret): {C2}')

# C1 and C2 should be the same shared secret


Q1 (Alice public): 5
Q2 (Bob public): 2
C1 (Alice shared secret): 3
C2 (Bob shared secret): 3


### Diffie Hellman

In [75]:
# Diffie-Hellman Key Exchange
# Generator G and prime modulus p
G = 5  # Generator
p = 23  # Prime modulus

# Private keys for two parties (Alice and Bob)
d1 = 6  # Alice's private key
d2 = 15  # Bob's private key

# Public keys (generated by each party)
Q1 = (G ** d1) % p  # Alice's public key
Q2 = (G ** d2) % p  # Bob's public key

print(f"Q1 (Alice's public key): {Q1}")
print(f"Q2 (Bob's public key): {Q2}")

# Each party computes the shared secret
# Alice uses Bob's public key, and Bob uses Alice's public key
shared_secret_Alice = (Q2 ** d1) % p  # Alice computes the shared secret
shared_secret_Bob = (Q1 ** d2) % p  # Bob computes the shared secret

print(f"Shared secret computed by Alice: {shared_secret_Alice}")
print(f"Shared secret computed by Bob: {shared_secret_Bob}")

# Both shared secrets should be the same


Q1 (Alice's public key): 8
Q2 (Bob's public key): 19
Shared secret computed by Alice: 2
Shared secret computed by Bob: 2


### El Gamal

In [76]:
# ElGamal Encryption
# Generator G and prime modulus p
G = 5  # Generator
p = 23  # Prime modulus

# Private key for recipient (Bob)
d = 15  # Bob's private key

# Bob's public key
Q = (G ** d) % p  # Bob's public key

# Message to encrypt (should be less than p)
M = 13  # Example message to encrypt

# Sender chooses a random value k
k = 7  # Random value for encryption

# ElGamal Encryption
C1 = (G ** k) % p  # Part 1 of the ciphertext
C2 = (M * (Q ** k)) % p  # Part 2 of the ciphertext

print(f"Ciphertext (C1, C2): ({C1}, {C2})")

# ElGamal Decryption
decrypted_message = (C2 * pow(C1, p - 1 - d, p)) % p

print(f"Decrypted message: {decrypted_message}")

Ciphertext (C1, C2): (17, 11)
Decrypted message: 13


In [42]:
# Example usage:
P = (0, 1)
k = 2
p = 5

steps = point_mul_add(P, k, p)

# Print steps
for step in steps:
    print(step)

('Double', (0, 4))


In [ ]:
# Example usage:
P = (0, 1)
k = 2
p = 5

steps = point_mul_add(P, k, p)

# Print steps
for step in steps:
    print(step)